In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
flights_sample_small=pd.read_csv('flights_sample_small.csv')

## 1.   knock-on Feature

In [19]:
X=flights_sample_small[['fl_date','tail_num','crs_dep_time','crs_arr_time']]

,fl_date,tail_num,crs_dep_time,crs_arr_time
0,2019-01-01,N576NN,620,956
1,2019-01-01,N575NN,2157,2311
2,2019-01-01,N254PS,1151,1325
3,2019-01-01,N254PS,1009,1126
4,2019-01-01,N710PS,2000,2134
...,...,...,...,...
19517,2019-01-01,N909DA,1903,2033
19518,2019-01-01,N344NW,1656,1857
19519,2019-01-01,N659DL,932,1532
19520,2019-01-01,N365DN,830,1047


In [22]:
X['crs_dep_time'] = X['crs_dep_time'].astype(str).str.zfill(4)
X['crs_arr_time'] = X['crs_arr_time'].astype(str).str.zfill(4)

def get_hour_min(x):    
    return str(int(x[:2]))+':'+str(int(x[-2:]))

X['crs_dep_time'] = X['crs_dep_time'].apply(lambda x: get_hour_min(x))
X['crs_arr_time'] = X['crs_arr_time'].apply(lambda x: get_hour_min(x))

#adding zero so that each time value consists of 4 digits
X['crs_dep_time'] = X['crs_dep_time'].str.replace('24:0', '00:00')
X['crs_arr_time'] = X['crs_arr_time'].str.replace('24:0', '00:00')


# convert time columns to datetime
X['crs_dep_time'] = pd.to_datetime(X['crs_dep_time'], format='%H:%M')
X['crs_arr_time'] = pd.to_datetime(X['crs_arr_time'], format='%H:%M')


In [23]:
X['knock-on']=np.nan
flights_sample_small_clean=X
flights_sample_small_clean

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,2019-01-01,N576NN,1900-01-01 06:20:00,1900-01-01 09:56:00,NaN
1,2019-01-01,N575NN,1900-01-01 21:57:00,1900-01-01 23:11:00,NaN
2,2019-01-01,N254PS,1900-01-01 11:51:00,1900-01-01 13:25:00,NaN
3,2019-01-01,N254PS,1900-01-01 10:09:00,1900-01-01 11:26:00,NaN
4,2019-01-01,N710PS,1900-01-01 20:00:00,1900-01-01 21:34:00,NaN
...,...,...,...,...,...
19517,2019-01-01,N909DA,1900-01-01 19:03:00,1900-01-01 20:33:00,NaN
19518,2019-01-01,N344NW,1900-01-01 16:56:00,1900-01-01 18:57:00,NaN
19519,2019-01-01,N659DL,1900-01-01 09:32:00,1900-01-01 15:32:00,NaN
19520,2019-01-01,N365DN,1900-01-01 08:30:00,1900-01-01 10:47:00,NaN


In [24]:
#unique tail nums and dates
tail_num_val=list(X['tail_num'].unique())
fl_date_val=list(X['fl_date'].unique())

In [25]:
#define a function to filter the table
def filtered_sorted(tail_num_val,fl_date_val):
    a=X[(X['tail_num'] == tail_num_val) & (X['fl_date'] == fl_date_val)].sort_values('crs_dep_time').reset_index()
    return a

In [26]:
aa=filtered_sorted('N576NN', '2019-01-01')

aa

,index,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,0,2019-01-01,N576NN,1900-01-01 06:20:00,1900-01-01 09:56:00,NaN
1,15521,2019-01-01,N576NN,1900-01-01 11:40:00,1900-01-01 13:25:00,NaN
2,15520,2019-01-01,N576NN,1900-01-01 13:55:00,1900-01-01 15:44:00,NaN
3,9015,2019-01-01,N576NN,1900-01-01 16:34:00,1900-01-01 17:38:00,NaN
4,9016,2019-01-01,N576NN,1900-01-01 18:08:00,1900-01-01 19:40:00,NaN
5,62,2019-01-01,N576NN,1900-01-01 20:21:00,1900-01-01 22:30:00,NaN


In [27]:
for i in tail_num_val:
    #print (i)
    for j in fl_date_val:
        #print (j)
        df=filtered_sorted(i,j)
        #print (df)
        for k in range(len(df)-1):
            a=df.loc[k+1, 'index'] #index_X
            X.loc[a,'knock-on'] #value of knock-on
            c=(df.loc[k+1, 'crs_dep_time'])
            d=(df.loc[k, 'crs_arr_time'])
            X.loc[a,'knock-on']=c-d
            X.loc[a,'knock-on']=X.loc[a,'knock-on']

In [47]:
#extract only the time part
# X['crs_dep_time']=X['crs_dep_time'].dt.time
# X['crs_arr_time']=X['crs_arr_time'].dt.time
type(X['knock-on'].iloc[1])

pandas._libs.tslibs.timedeltas.Timedelta

In [45]:
flights_sample_small_knock_on=X
X

,fl_date,tail_num,crs_dep_time,crs_arr_time,knock-on
0,2019-01-01,N576NN,06:20:00,09:56:00,NaN
1,2019-01-01,N575NN,21:57:00,23:11:00,0 days 00:30:00
2,2019-01-01,N254PS,11:51:00,13:25:00,0 days 00:25:00
3,2019-01-01,N254PS,10:09:00,11:26:00,0 days 01:01:00
4,2019-01-01,N710PS,20:00:00,21:34:00,0 days 00:40:00
...,...,...,...,...,...
19517,2019-01-01,N909DA,19:03:00,20:33:00,0 days 00:50:00
19518,2019-01-01,N344NW,16:56:00,18:57:00,0 days 00:46:00
19519,2019-01-01,N659DL,09:32:00,15:32:00,NaN
19520,2019-01-01,N365DN,08:30:00,10:47:00,NaN


In [39]:
X.to_csv('./tables/flights_sample_small_knock_on.csv', index=False)